In [28]:
import open3d as o3d
import os
import numpy as np
from sklearn.neighbors import kneighbors_graph
import maxflow
import matplotlib.pyplot as plt

In [29]:
datasetPath = "/PublicData/CadDataset/a1.0.0_10"
part = "139661_105bb47a"#"140294_f659b875"
name = "assembly.obj"
file_path = os.path.join(datasetPath,part,name)

In [30]:
# 读取网格
mesh = o3d.io.read_triangle_mesh(file_path)
if not mesh.has_vertices():
    raise RuntimeError("网格无顶点")
points = np.asarray(mesh.vertices)
print(f"读取网格顶点数: {len(points)}")

读取网格顶点数: 13990


In [31]:
def construct_graph(points, k=8):
    # 构造邻接图，返回 COO格式的行列索引和边权重
    connectivity = kneighbors_graph(points, n_neighbors=k, mode='connectivity', include_self=False).tocoo()
    row, col = connectivity.row, connectivity.col
    edges = []
    weights = []
    for i, j in zip(row, col):
        dist = np.linalg.norm(points[i] - points[j])
        weight = np.exp(-dist * 10)  # 距离越近权重越大
        edges.append((i, j))
        weights.append(weight)
    return edges, weights

In [32]:
edges, weights = construct_graph(points, 16)

In [33]:
def compute_data_cost(points, num_labels):
    # 简单模拟数据项，基于Z值离散成多个类别的距离作为代价
    # 实际可替换为你的先验置信度等
    z = points[:, 2]
    z_min, z_max = z.min(), z.max()
    labels_pos = np.linspace(z_min, z_max, num_labels)
    data_cost = np.zeros((len(points), num_labels))
    for i in range(num_labels):
        data_cost[:, i] = np.abs(z - labels_pos[i])
    # 归一化
    data_cost /= data_cost.max()
    return data_cost

In [34]:
data_cost = compute_data_cost(points, num_labels=4)

In [35]:
def alpha_expansion(points, edges, weights, data_cost, max_iter=10):
    N, num_labels = data_cost.shape
    # 初始化标签（比如全部归0）
    labels = np.zeros(N, dtype=np.int32)

    for it in range(max_iter):
        print(f"迭代 {it+1}/{max_iter}")
        changed = False
        for alpha in range(num_labels):
            # 构造二分类图割问题，标签为 alpha 和 当前标签
            g = maxflow.Graph[float]()
            nodes = g.add_nodes(N)

            # 添加终端边（data term）
            for i in range(N):
                if labels[i] == alpha:
                    g.add_tedge(i, data_cost[i, alpha], float('inf'))  # 保持alpha标签的点不被改
                else:
                    g.add_tedge(i, data_cost[i, alpha], data_cost[i, labels[i]])

            # 添加邻边（smoothness term）
            for (i, j), w in zip(edges, weights):
                if labels[i] == labels[j]:
                    cap_same = 0
                else:
                    cap_same = 1  # 简单平滑项，惩罚不同标签邻接
                # 对α扩展构造边权重
                cap_i = 1 if labels[i] != alpha else 0
                cap_j = 1 if labels[j] != alpha else 0
                weight_ij = w

                # 边权重对应下面几种情况：
                # i标签为alpha, j标签为alpha: 不存在，因为一个点只能选一个标签
                # 这里简单设置边容量为 weight_ij * (cap_i + cap_j)
                g.add_edge(i, j, weight_ij * cap_i, weight_ij * cap_j)

            flow = g.maxflow()
            # 获取二分类结果
            new_labels = np.array([alpha if g.get_segment(i) == 0 else labels[i] for i in range(N)])

            if not np.array_equal(new_labels, labels):
                changed = True
                labels = new_labels

        if not changed:
            print("收敛，提前退出")
            break

    return labels

In [36]:
labels = alpha_expansion(points, edges, weights, data_cost, max_iter=10)
print("分割类别数:", labels.max() + 1)

迭代 1/10
迭代 2/10
收敛，提前退出
分割类别数: 4


In [37]:
# 顶点染色
num_clusters = labels.max() + 1
colors = np.zeros((len(labels), 3))
cmap = plt.get_cmap("tab10")
for i in range(num_clusters):
    colors[labels == i] = cmap(i / max(1, num_clusters - 1))[:3]
mesh.vertex_colors = o3d.utility.Vector3dVector(colors)

In [38]:
o3d.io.write_triangle_mesh("graphcut_alpha_segment2_k=16.ply",mesh)

True